In [5]:
# Train for sequence of images

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# Set parameters
img_width, img_height = 224, 224
batch_size = 32

# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))  # Using 3 channels here

# Freeze the base model
base_model.trainable = False

# Define the custom head for our network
model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



# Data generators for training and validation
# Note: Using `preprocess_input` from MobileNetV2 for preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Preprocess suitable for MobileNetV2
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/home/bule/projects/Dice/data/eyes2',  # Change to your data directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Changed to 'rgb'
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/home/bule/projects/Dice/data/eyes2',  # Change to your data directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Changed to 'rgb'
    subset='validation')

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10  # Adjust the number of epochs
)



Found 289 images belonging to 3 classes.
Found 71 images belonging to 3 classes.
Epoch 1/10
9/9 [==============================] - 3s 240ms/step - loss: 1.2384 - accuracy: 0.6809 - val_loss: 0.6308 - val_accuracy: 0.8281
Epoch 2/10
9/9 [==============================] - 2s 189ms/step - loss: 0.2903 - accuracy: 0.8755 - val_loss: 0.2313 - val_accuracy: 0.8750
Epoch 3/10
9/9 [==============================] - 2s 191ms/step - loss: 0.1854 - accuracy: 0.9105 - val_loss: 0.2778 - val_accuracy: 0.9062
Epoch 4/10
9/9 [==============================] - 2s 189ms/step - loss: 0.1194 - accuracy: 0.9416 - val_loss: 0.3001 - val_accuracy: 0.9062
Epoch 5/10
9/9 [==============================] - 2s 190ms/step - loss: 0.1134 - accuracy: 0.9494 - val_loss: 0.2865 - val_accuracy: 0.9062
Epoch 6/10
9/9 [==============================] - 2s 188ms/step - loss: 0.1243 - accuracy: 0.9455 - val_loss: 0.3103 - val_accuracy: 0.9062
Epoch 7/10
9/9 [==============================] - 2s 188ms/step - loss: 0.1227 

In [2]:
# Train for sequence of images
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# Set parameters
img_width, img_height = 224, 224
batch_size = 32

# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))  # Using 3 channels here

# Freeze the base model
base_model.trainable = False

# Define the custom head for our network
model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Assuming 3 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Function to add noise
def add_noise(img):
    '''Add random noise to an image'''
    row, col, ch = img.shape
    s_vs_p = 0.5
    amount = 0.04
    out = np.copy(img)
    # Salt mode
    num_salt = np.ceil(amount * img.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in img.shape]
    out[coords] = 1

    # Pepper mode
    num_pepper = np.ceil(amount* img.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in img.shape]
    out[coords] = 0
    return out

# Function to add blur
def add_blur(img):
    '''Add random blur to an image'''
    return cv2.GaussianBlur(img, (5, 5), 0)

# Custom preprocessing function
def custom_preprocess_input(img):
    img = preprocess_input(img)  # preprocess suitable for MobileNetV2
    # img = add_noise(img)  # Add random noise
    #img = add_blur(img)  # Add random blur
    return img
# Create an instance of the ImageDataGenerator class
train_datagen = ImageDataGenerator(
    shear_range=15,  # Shear angle in counter-clockwise direction as radians
    preprocessing_function=custom_preprocess_input,  # Custom preprocess function
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
    '/home/bule/projects/Dice/data/eyes2/states',  # Change to your data directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Changed to 'rgb'
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/home/bule/projects/Dice/data/eyes2/states',  # Change to your data directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Changed to 'rgb'
    subset='validation')

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10  # Adjust the number of epochs
)



FileNotFoundError: [Errno 2] No such file or directory: '/home/bule/projects/Dice/data/eyes2/states'

In [6]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model_dices_eyes2.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TensorFlow Lite and saved.")


INFO:tensorflow:Assets written to: /tmp/tmp0rqnxsx0/assets


INFO:tensorflow:Assets written to: /tmp/tmp0rqnxsx0/assets


Model converted to TensorFlow Lite and saved.


2023-11-04 20:25:25.372475: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-04 20:25:25.372492: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-04 20:25:25.372620: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0rqnxsx0
2023-11-04 20:25:25.385446: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-04 20:25:25.385458: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp0rqnxsx0
2023-11-04 20:25:25.419133: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-04 20:25:25.627811: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp0rqnxsx0
2023-11-04 20:25:25.706321: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 333700 

In [ ]:
# Train for dices

In [5]:
# Train for sequence of images

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# Set parameters
img_width, img_height = 224, 224
batch_size = 32

# Load the MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))  # Using 3 channels here

# Freeze the base model
base_model.trainable = False

# Define the custom head for our network
model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Assuming 3 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data generators for training and validation
# Note: Using `preprocess_input` from MobileNetV2 for preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Preprocess suitable for MobileNetV2
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/home/bule/projects/Dice/data_still_numbers',  # Change to your data directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Changed to 'rgb'
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    '/home/bule/projects/Dice/data_still_numbers',  # Change to your data directory
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Changed to 'rgb'
    subset='validation')

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=40  # Adjust the number of epochs
)



Found 141 images belonging to 6 classes.
Found 32 images belonging to 6 classes.
Epoch 1/40
4/4 [==============================] - 2s 291ms/step - loss: 2.8328 - accuracy: 0.2844 - val_loss: 2.1592 - val_accuracy: 0.3438
Epoch 2/40
4/4 [==============================] - 1s 162ms/step - loss: 2.0232 - accuracy: 0.3486 - val_loss: 1.8857 - val_accuracy: 0.2500
Epoch 3/40
4/4 [==============================] - 1s 211ms/step - loss: 1.4730 - accuracy: 0.5138 - val_loss: 2.0892 - val_accuracy: 0.2812
Epoch 4/40
4/4 [==============================] - 1s 171ms/step - loss: 1.5773 - accuracy: 0.5046 - val_loss: 1.8166 - val_accuracy: 0.3750
Epoch 5/40
4/4 [==============================] - 1s 191ms/step - loss: 1.3417 - accuracy: 0.5312 - val_loss: 1.9780 - val_accuracy: 0.2812
Epoch 6/40
4/4 [==============================] - 1s 169ms/step - loss: 1.2462 - accuracy: 0.5046 - val_loss: 1.9321 - val_accuracy: 0.2500
Epoch 7/40
4/4 [==============================] - 1s 186ms/step - loss: 1.0309 

In [6]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model.tflite_classifierdices', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TensorFlow Lite and saved.")


INFO:tensorflow:Assets written to: /tmp/tmphl8bjbo4/assets


INFO:tensorflow:Assets written to: /tmp/tmphl8bjbo4/assets


Model converted to TensorFlow Lite and saved.


2023-11-02 17:13:09.152198: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-02 17:13:09.152215: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-02 17:13:09.152344: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmphl8bjbo4
2023-11-02 17:13:09.162252: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-02 17:13:09.162264: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmphl8bjbo4
2023-11-02 17:13:09.193014: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-02 17:13:09.399998: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmphl8bjbo4
2023-11-02 17:13:09.477801: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 325457 

run on raspy with tflite 

In [ ]:
import cv2
import numpy as np
import tflite_runtime.interpreter as tflite

# Define class labels for the first and second models
class_labels_state = ["empty", "rolling", "still"]
class_labels_dice = ["1", "2", "3", "4", "5", "6"]

# Load the TFLite models
interpreter_state = tflite.Interpreter(model_path="model2.tflite")
interpreter_state.allocate_tensors()

interpreter_dice = tflite.Interpreter(model_path="model_classifierdices.tflite")
interpreter_dice.allocate_tensors()

# Get model details
input_details_state = interpreter_state.get_input_details()
output_details_state = interpreter_state.get_output_details()
input_shape_state = input_details_state[0]['shape']

input_details_dice = interpreter_dice.get_input_details()
output_details_dice = interpreter_dice.get_output_details()
input_shape_dice = input_details_dice[0]['shape']

# Open a handle to the default webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Convert the frame to RGB, then resize and preprocess it
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (224, 224))  # Assuming both models use the same input size
    input_data = np.expand_dims(frame_resized, axis=0)
    if input_details_state[0]['dtype'] == np.float32: 
        input_data = (np.float32(input_data) - 127.5) / 127.5

    # Feed the frame to the state model
    interpreter_state.set_tensor(input_details_state[0]['index'], input_data)
    interpreter_state.invoke()

    # Retrieve the results and determine the class with the highest probability
    output_data_state = interpreter_state.get_tensor(output_details_state[0]['index'])
    prediction_state = np.argmax(output_data_state)
    class_label_state = class_labels_state[prediction_state]









    # Run the dice model only if the state is "still"
    dice_label = ""
    if class_label_state == "still":
        interpreter_dice.set_tensor(input_details_dice[0]['index'], input_data)
        interpreter_dice.invoke()
        output_data_dice = interpreter_dice.get_tensor(output_details_dice[0]['index'])
        prediction_dice = np.argmax(output_data_dice)
        dice_label = class_labels_dice[prediction_dice]

    # Display the resulting frame with the class label
    cv2.putText(frame, f'State: {class_label_state}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    if dice_label:
        cv2.putText(frame, f'Dice: {dice_label}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()